In [ ]:
# # Encoding Ising-like Hamiltonians as quantum circuits
#
# A more lengthy explanation is given in
# https://www.mustythoughts.com/variational-quantum-eigensolver-explained
using LinearAlgebra: I, tr

In [2]:
# Computational basis
𝟎 = [1,0]
𝟏 = [0,1]

2-element Array{Int64,1}:
 0
 1

In [3]:
X = [0 1; 1 0]
Z = [1 0; 0 -1] # we measure in Z basis

2×2 Array{Int64,2}:
 1   0
 0  -1

In [4]:
H = [1 1; 1 -1]/√2
p = H * 𝟎 # = |+⟩
m = H * 𝟏 # = |-⟩

2-element Array{Float64,1}:
  0.7071067811865475
 -0.7071067811865475

## Measurements in fixed Z-basis: rotate to measure other Pauli matrices

If we measure in the Z basis its not possible to distinguish between |+⟩ and
|-⟩. 

In [5]:
@info "⟨+|Z|+⟩ ≈ ⟨-|Z|-⟩", p'*Z*p ≈ m'* Z*m

┌ Info: ("⟨+|Z|+⟩ ≈ ⟨-|Z|-⟩", true)
└ @ Main In[5]:1


However, if we rotate the basis, then it is possible.

In [6]:
Ry(θ) = [cos(θ/2) -sin(θ/2); sin(θ/2) cos(θ/2)]

Ry (generic function with 1 method)

In [12]:
@assert Ry(π/2)*p ≈ 𝟏
@assert Ry(π/2)*m ≈ 𝟎

In [8]:
p_rot = Ry(-π/2)*p
m_rot = Ry(-π/2)*m
@info "⟨+|Z|Ry(-π/2)|+⟩ ≈ ⟨-|Z|Ry(-π/2)|-⟩", p_rot'*Z*p_rot ≈ m_rot'* Z*m_rot
@info "Outcome of ⟨+|Z|Ry(-π/2)|+⟩ = ", p_rot'*Z*p_rot
@info "Outcome of ⟨-|Z|Ry(-π/2)|-⟩ = ", m_rot'* Z*m_rot

┌ Info: ("⟨+|Z|Ry(-π/2)|+⟩ ≈ ⟨-|Z|Ry(-π/2)|-⟩", false)
└ @ Main In[8]:3
┌ Info: ("Outcome of ⟨+|Z|Ry(-π/2)|+⟩ = ", 0.9999999999999998)
└ @ Main In[8]:4
┌ Info: ("Outcome of ⟨-|Z|Ry(-π/2)|-⟩ = ", -0.9999999999999998)
└ @ Main In[8]:5


The solution is to apply
* $R_y(−\pi/2)$      if Hamiltonian has $\hat{X}$
* $R_x(\pi/2)$       if Hamiltonian has $\hat{Y}$
* $I$                if Hamiltonian has $\hat{Z}$

## Divide the Hamiltonian in single Pauli terms

In [9]:
𝓗₁ = 2Z
𝓗₂ = X
𝓗₃ = I(2)
𝓗 = 𝓗₁ + 𝓗₂ + 𝓗₃
@assert 𝓗 ≈ [3 1; 1 -1]

## Initial state

We use a parametrized initial wavefunction for reasons that will be explained
later but you can already guess that we will do a variational search.

In [10]:
# Ansatz will be Ry(θ)*𝟎
ansatz(θ) = Ry(θ)*𝟎

ansatz (generic function with 1 method)

## A circuit for each term

In [13]:
for θ in [0,π]

   @info "θ = $θ"
   # Circuit for 𝓗₁: ψ ---[ h₁ )===

   ψ = ansatz(θ)

   h₁ = 2ψ' * Z * ψ
   @info "  E₁ = $h₁"

   # Circuit for 𝓗₂: ψ ---[ Ry ]---[ h₂ )===

   ψ = Ry(-π/2)*ansatz(θ)

   h₂ = ψ' * Z * ψ
   @info "  E₂ = $h₂"

   # Circuit for 𝓗₃:

   h₃ = 1
   @info "  E₃ = $h₃"

   @info "  ⟨ψ($θ)|𝓗|ψ($θ)⟩ = $(h₁ + h₂ + h₃)"
end

┌ Info: θ = 0.0
└ @ Main In[13]:3
┌ Info:   E₁ = 2.0
└ @ Main In[13]:9
┌ Info:   E₂ = 2.220446049250313e-16
└ @ Main In[13]:16
┌ Info:   E₃ = 1
└ @ Main In[13]:21
┌ Info:   ⟨ψ(0.0)|𝓗|ψ(0.0)⟩ = 3.0
└ @ Main In[13]:23
┌ Info: θ = 3.141592653589793
└ @ Main In[13]:3
┌ Info:   E₁ = -2.0
└ @ Main In[13]:9
┌ Info:   E₂ = -2.220446049250313e-16
└ @ Main In[13]:16
┌ Info:   E₃ = 1
└ @ Main In[13]:21
┌ Info:   ⟨ψ(3.141592653589793)|𝓗|ψ(3.141592653589793)⟩ = -1.0
└ @ Main In[13]:23
